In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Problem Statement 1: Linear Regression: Diamond price prediction.¶**
You are hired by a company Gem Stones co ltd, which is a cubic zirconia manufacturer. You are provided with the dataset containing the prices and other attributes of almost 27,000 cubic zirconia (which is an inexpensive diamond alternative with many of the same qualities as a diamond). The company is earning different profits on different prize slots. You have to help the company in predicting the price for the stone on the bases of the details given in the dataset so it can distinguish between higher profitable stones and lower profitable stones so as to have better profit share. Also, provide them with the best 5 attributes that are most important.

Data Dictionary:
Variable Name : Description

Carat:Carat weight of the cubic zirconia.
Cut: Describe the cut quality of the cubic zirconia. Quality is increasing order Fair, Good, Very Good,Premium,Ideal.
Color: Colour of the cubic zirconia.With D being the best and J the worst.
Clarity :cubic zirconia Clarity refers to the absence of the Inclusions and Blemishes. (In order from Best to Worst, FL = 1. flawless, I3= level 3 inclusions) FL, IF, VVS1, VVS2, VS1, VS2, SI1, SI2, I1, I2, I3
Depth : The Height of a cubic zirconia, measured from the Culet to the table, divided by its average Girdle Diameter.
Table : The Width of the cubic zirconia's Table expressed as a Percentage of its Average Diameter.
Price : the Price of the cubic zirconia.
X : Length of the cubic zirconia in mm.
Y : Width of the cubic zirconia in mm.
Z : Height of the cubic zirconia in mm.

In [ ]:
import numpy as np   
import pandas as pd    
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt   
import matplotlib.style
%matplotlib inline 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import math

1.1. Read the data and do exploratory data analysis. Describe the data briefly. (Check the null values, Data types, shape, EDA). Perform Univariate and Bivariate Analysis.¶

**Importing data**

In [ ]:
# reading the CSV file into pandas dataframe
df = pd.read_csv('../input/gemstone-price-prediction/cubic_zirconia.csv')

In [ ]:
# Check top few records to get a feel of the data structure
df.head(10)

In [ ]:
df.columns

In [ ]:
df.dtypes  # Checking the type of the dataset.

In [ ]:
df.shape  # Checking the shape of the dataset.

In [ ]:
#Getting the info data types column wise
df.info()

### Observation-1: 
1. <b>.The data set contains 26967 row, 11 columns .
1. <b>.In the given data set there are 2 Integer type features,6 Float type features.
   <b> 3 Object type features. Where 'price' is the target variable and all other are predector variable.
1. <b>The first column is an index ("Unnamed: 0")as this only serial no, we can remove it.
1. <b>Exept depth, in all the column non null count is 26967.

In [ ]:
# drop the serial no column as it is useless for the model
lr_df = df.drop('Unnamed: 0', axis=1)

In [ ]:
lr_df.columns

In [ ]:
# Getting the summary statistics of the dataset
lr_df.describe().T

In [ ]:
# Getting the summary statistics of the  object variable.
lr_df.describe(include=['object'])

In [ ]:
# Checking for the values which are equal to zero.
print("Number of rows with x == 0: {} ".format((lr_df.x==0).sum()))
print("Number of rows with y == 0: {} ".format((lr_df.y==0).sum()))
print("Number of rows with z == 0: {} ".format((lr_df.z==0).sum()))
print("Number of rows with depth == 0: {} ".format((lr_df.depth==0).sum()))

In [ ]:
lr_df.shape

In [ ]:
#Dropping dimentionless diamonds
lr_df = lr_df.drop(lr_df[lr_df["x"]==0].index)
lr_df = lr_df.drop(lr_df[lr_df["y"]==0].index)
lr_df = lr_df.drop(lr_df[lr_df["z"]==0].index)
lr_df.shape

**Observation-2:**
.On the given data set the the mean and median values does not have much differenc.
.We can observe Min value of "x", "y", & "z" are zero this indicates that they are faulty values. As we   know dimensionless or 2-dimensional diamonds are not possible. So we need to filter out those as it    clearly faulty data entries.
.There are three object data type 'cut', 'color' and 'clarity'.

### Performing EDA :  We will follow the below mentioned steps to perform EDA

#Step 1 :Checking & Removing duplicates.

#Step 2: Checking and treating Missing value.

#Step 3: Outlier Treatment.

#Step 4: Univariate  Analysis.

#Step 5: Bivariate Analysis.

### EDA-Step-1: Checking for duplicate records in the data.

In [ ]:
dups = lr_df.duplicated()
print('Number of duplicate rows = %d' % (dups.sum()))
print(lr_df.shape)

In [ ]:
print('Before',lr_df.shape)
lr_df.drop_duplicates(inplace=True) 
print('After',lr_df.shape)

In [ ]:
dups = lr_df.duplicated()
print('Number of duplicate rows = %d' % (dups.sum()))

### EDA-Step 2:  Checking Missing value.

In [ ]:
# Are there any missing values ?
lr_df.isnull().sum()

#### Observation-3: 
we can observe there are 697 missing value in the depth column.
Missing value treatment will be done in section 1.2.

### EDA-Step 3 : Outlier Checks.

In [ ]:
cols = ['carat','depth', 'table', 'x', 'y', 'z',
       'price' ]
for i in cols:
    sns.boxplot(lr_df[i],whis=1.5)
    plt.show();

In [ ]:
### Outlier treatment :
def remove_outlier(col):
    sorted(col)
    Q1,Q3=np.percentile(col,[25,75])
    IQR=Q3-Q1
    lower_range= Q1-(1.5 * IQR)
    upper_range= Q3+(1.5 * IQR)
    return lower_range, upper_range

In [ ]:
for column in lr_df[cols].columns:
    lr,ur=remove_outlier(lr_df[column])
    lr_df[column]=np.where(lr_df[column]>ur,ur,lr_df[column])
    lr_df[column]=np.where(lr_df[column]<lr,lr,lr_df[column])

In [ ]:
cols = ['carat','depth', 'table', 'x', 'y', 'z',
       'price' ]
for i in cols:
    sns.boxplot(lr_df[i],whis=1.5)
    plt.show();

**EDA-Step 4 : Univariate Analysis.**

In [ ]:
lr_df.hist(figsize=(20,30));

In [ ]:
lr_df.skew()   # to measure the skeweness of every attribute.

#### Observation-4: :
 (1).There is significant amount of outlier present in some variable.
 (2) We can see that the distribution of some quantitative features like "carat" and the target feature "price" are heavily "right-skewed".

**EDA-Step 5 : Bivariate Analysis.**

In [ ]:
import seaborn as sns

df_attr = ( lr_df[cols])
sns.pairplot(df_attr, diag_kind='kde')  
plt.show()

**Get the Correlation Heatmap**

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(lr_df.corr(),annot=True,fmt='.2f',cmap='rainbow',mask=np.triu(lr_df.corr(),+1))
plt.show()

**How each feature affects the price of diamonds.¶**

In [ ]:
correlations = lr_df.corr()
correlations["price"].sort_values(ascending=False)

#### Observation-5: 
It can be inferred that most features correlate with the price of Diamond. The notable exception is "depth" which has a negligible correlation (<1%).

### EDA for Categorical variable.

In [ ]:
# EDA for categorical columns 'CUT'.
sns.catplot('cut', data=lr_df, kind='count',aspect=2.5)

In [ ]:
sns.catplot(x='cut', y='price', kind='box', data=lr_df, aspect=2.5)

bservation on 'CUT': The Premium Cut on Diamonds are the most Expensive, followed by Very Good Cut.

In [ ]:
# EDA for categorical columns 'Color'.
sns.catplot('color', kind='count', data=lr_df, aspect=2.5)

In [ ]:
sns.catplot(x='color', y='price', data=lr_df, aspect =2.5, kind='box')

In [ ]:
# EDA for categorical columns 'Clarity'.
sns.catplot('clarity', data=lr_df, kind='count',aspect=2.5)

In [ ]:
sns.catplot(x='clarity', y='price', data=lr_df, aspect =2.5, kind='box')

**Observation on 'clarity': The Diamonds clarity with VS1 & VS2  are the most Expensive.**

### The inferences drawn from the above Exploratory Data analysis: 

#### Observation-1: 
(1).'Price' is the target variable while all others are the predictors.
(2).The data set contains 26967 row, 11 column.
(3).In the given data set there are 2 Integer type features,6 Float type features.
    3 Object type features. Where 'price' is the target variable and all other are predector variable.
(4)The first column is an index ("Unnamed: 0")as this only serial no, we can remove it.

**Observation-2:**
(1).On the given data set the the mean and median values does not have much differenc. (2).We can observe Min value of "x", "y", "z" are zero this indicates that they are faulty values. As we know dimensionless or 2-dimensional diamonds are not possible. So we have filter out those as it clearly faulty data entries. (3).There are three object data type 'cut', 'color' and 'clarity'.

**Observation-3:**
we can observe there are 697 missing value in the depth column. There are some duplicate row present. (33 duplicate rows out of 26958). which is nearly 0.12 % of the total data. So on this case we have dropped the duplicated row.

#### Observation-4: :
There are significant amount of outlier present in some variable,the features with datapoint that are far from the rest of dataset which will affect the outcome of our regression model. So we have treat the outliar.
We can see that the distribution of some quantitative features like "carat" and the target feature "price" are heavily "right-skewed".

#### Observation-5: 
It looks like most features do correlate with the price of Diamond. The notable exception is "depth" which has a negligble correlation (~1%).
Observation on 'CUT': The Premium Cut on Diamonds are the most Expensive, followed by Very Good Cut.

### 1.2 Impute null values if present, also check for the values which are equal to zero. Do they have any meaning or do we need to change them or drop them? Do you think scaling is necessary in this case?

In [ ]:
# As we have checked there Are some missing values.
lr_df.isnull().sum()

In [ ]:
lr_df.median()

### Imputing missing values

In [ ]:
#lets replace the missing values with median value. 
lr_df = lr_df.fillna(lr_df.median())

In [ ]:
lr_df.isnull().sum()

### Checking for the values which are equal to zero.

In Qs.1.1 we have alrady check for 'Zero' value.
and  we can observe there are some amount of 'Zero' value present on the data set on variable 'x', 'y','z'.

This indicates that they are faulty values. 
### As we know dimensionless or 2-dimensional diamonds are not possible. So we have filter out those as it clearly faulty data entries.

In [ ]:
lr_df.shape # after removing 'zero value' from data set the data shape became as follows.

### Do you think scaling is necessary in this case?

Scaling or Standardizing the features around the centre and 0 with a standard deviation of 1 is important when we compare measurements that have different units. Variables that are measured at different scales do not contribute equally to the analysis and might end up creating a bias.

For example, A variable that ranges between 0 and 1000 will outweigh a variable that ranges between 0 and 1. Using these variables without standardization will give the variable with the larger range weight of 1000 in the analysis. Transforming the data to comparable scales can prevent this problem.

In this data set we can see the all the variable are in different scale i.e price are in 1000s unit and depth and table are in 100s unit, and carat is in 10s. So its necessary to scale or standardise the data to allow each variable to be compared on a common scale. With data measured in different "units" or on different scales (as here with different means and variances) this is an important data processing step if the results are to be meaningful or not dominated by the variables that have large variances.

### But is scaling  necessary in this case? 
No, it is not necessary, we'll get an equivalent solution whether we apply some kind of linear scaling or not. 
But recommended for regression techniques as well because it would help gradient descent to converge fast and reach the global minima. When number of features becomes large, it helps is running model quickly else the starting point would be very far from minima, if the scaling is not done in preprocessing.

<b>For now we will process the model without scaling and later we will check the output with scaled data of regression model output.

### 1.3 Encode the data (having string values) for Modelling. Data Split: Split the data into  train and test (70:30). Apply Linear regression. Performance Metrics: Check the performance of Predictions on Train and Test sets using Rsquare, RMSE.

### Encode the data (having string values)

#### Geting unique counts of all Objects.

In [ ]:
print('cut\n',lr_df.cut.value_counts())
print('\n')
print('color\n',lr_df.color.value_counts())
print('\n')
print('clarity\n',lr_df.clarity.value_counts())
print('\n')

### Converting objects to categorical codes.

In [ ]:
lr_df['cut']=np.where(lr_df['cut'] =='Ideal', '4', lr_df['cut'])
lr_df['cut']=np.where(lr_df['cut'] =='Premium', '3', lr_df['cut'])
lr_df['cut']=np.where(lr_df['cut'] =='Very Good', '2', lr_df['cut'])
lr_df['cut']=np.where(lr_df['cut'] =='Good', '1', lr_df['cut'])
lr_df['cut']=np.where(lr_df['cut'] =='Fair', '0', lr_df['cut'])

In [ ]:
lr_df['color']=np.where(lr_df['color'] =='D', '6', lr_df['color'])
lr_df['color']=np.where(lr_df['color'] =='E', '5', lr_df['color'])
lr_df['color']=np.where(lr_df['color'] =='F', '4', lr_df['color'])
lr_df['color']=np.where(lr_df['color'] =='G', '3', lr_df['color'])
lr_df['color']=np.where(lr_df['color'] =='H', '2', lr_df['color'])
lr_df['color']=np.where(lr_df['color'] =='I', '1', lr_df['color'])
lr_df['color']=np.where(lr_df['color'] =='J', '0', lr_df['color'])

In [ ]:
lr_df['clarity']=np.where(lr_df['clarity'] =='IF', '7', lr_df['clarity'])
lr_df['clarity']=np.where(lr_df['clarity'] =='VVS1', '6', lr_df['clarity'])
lr_df['clarity']=np.where(lr_df['clarity'] =='VVS2', '5', lr_df['clarity'])
lr_df['clarity']=np.where(lr_df['clarity'] =='VS1', '4', lr_df['clarity'])
lr_df['clarity']=np.where(lr_df['clarity'] =='VS2', '3', lr_df['clarity'])
lr_df['clarity']=np.where(lr_df['clarity'] =='SI1', '2', lr_df['clarity'])
lr_df['clarity']=np.where(lr_df['clarity'] =='SI2', '1', lr_df['clarity'])
lr_df['clarity']=np.where(lr_df['clarity'] =='I1', '0', lr_df['clarity'])

In [ ]:
lr_df.dtypes # The'cut','color','clarity' column still showing as 'Object'.

In [ ]:
# converting the'cut','color','clarity' column from object / string type to float.

lr_df['cut'] = lr_df['cut'].astype('float64')
lr_df['color'] = lr_df['color'].astype('float64')
lr_df['clarity'] = lr_df['clarity'].astype('float64')

In [ ]:
lr_df.dtypes

In [ ]:
lr_df.head(10)

#### Train-Test Split:

In [ ]:
# Copy all the predictor variables into X dataframe
X = lr_df.drop('price', axis=1)

# Copy target into the y dataframe.This is the dependent variable
y = lr_df[['price']]
X.head()

In [ ]:
#Let us break the X and y dataframes into training set and test set. For this we will use
#Sklearn package's data splitting function which is based on random function

from sklearn.model_selection import train_test_split

In [ ]:
# Split X and y into training and test set in 70:30 ratio

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30 , random_state=1)

In [ ]:
# invoke the LinearRegression function and find the bestfit model on training data

regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

In [ ]:
# Let us explore the coefficients for each of the independent attributes

for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

In [ ]:
### Observation-1: 
Y=mx +c (m= m1,m2,m3...m9) here 9 diferent co-efficients will learn aling with the intercept which is "c" from the model.

From the above coefficients for each of the independent attributes we can conclude
The one unit increase in carat increases price by 8901.941.
The one unit increase in cut increases price by 109.188.
The one unit increase in color increases price by 272.921.
The one unit increase in clarity increases price by 436.441.
The one unit increase in y increases price by 1464.827.
The one unit increase in depth increases price by 8.236,

But The one unit increase in table decreases price by -17.345,
The one unit increase in x decreases price by -1417.908,
The one unit increase in z decreases price by -711.225.

In [ ]:
# Let us check the intercept for the model

intercept = regression_model.intercept_[0]

print("The intercept for our model is {}".format(intercept))

**Observation-2:**
The intercept (often labelled the constant) is the expected mean value of Y when all X=0. If X never equals 0, then the intercept has no intrinsic meaning.

The intercept for our model is -3171.950447307667. In preset case when the other predictor variable are zero i.e like carat,cut, color, clarity all are zero then the C=-3172. ( Y = m1X1 + m2X2+ ….. + mnXn + C + e) that means price is -3172. which is meaningless. We can do Z score or scaling the data and make it nearly zero.

In [ ]:
 # R square on training data
regression_model.score(X_train, y_train)

In [ ]:
# Model score - R2 or coeff of determinant
# R^2=1–RSS / TSS =  RegErr / TSS

# R square on testing data
regression_model.score(X_test, y_test)

### Observation-3: 
R-square is the percentage of the response variable variation that is explained by a linear model. Or:

R-square = Explained variation / Total variation

R-squared is always between 0 and 100%: 0% indicates that the model explains none of the variability of the response data around its mean.100% indicates that the model explains all the variability of the response data around its mean.
In this regression model we can see the R-square value on Training and Test data respectively 0.9311935886926559 and 0.931543712584074.

In [ ]:
#RMSE on Training data
predicted_train=regression_model.fit(X_train, y_train).predict(X_train)
np.sqrt(metrics.mean_squared_error(y_train,predicted_train))

In [ ]:
#RMSE on Testing data
predicted_test=regression_model.fit(X_train, y_train).predict(X_test)
np.sqrt(metrics.mean_squared_error(y_test,predicted_test))

In [ ]:
# Since this is regression, plot the predicted y value vs actual y values for the test data
# A good model's prediction will be close to actual leading to high R and R2 values
y_pred = regression_model.predict(X_test)
plt.scatter(y_test['price'], y_pred)

### Observation-4:
we can see that the is a linear plot, very strong corelation between the predicted y and actual y.
But there are lots of spread. That indicated some kind noise present on the data set i.e Unexplained variances on the output.

**Linear regression Performance Metrics:**

*  intercept for the model:     -3171.950447307667
*  R square on training data: 0.9311935886926559
*  R square on testing data:   0.931543712584074
* 
*  RMSE on Training data:      907.1312415459143
*  RMSE on Testing data:        911.8447345328436


 As the training data & testing data score are almost inline,  we can conclude this model is a Right-Fit Model.


### Applying zscore statsmodels

In [ ]:
from scipy.stats import zscore

X_train_scaled  = X_train.apply(zscore)
X_test_scaled = X_test.apply(zscore)
y_train_scaled = y_train.apply(zscore)
y_test_scaled = y_test.apply(zscore)

In [ ]:
# invoke the LinearRegression function and find the bestfit model on training data

regression_model = LinearRegression()
regression_model.fit(X_train_scaled, y_train_scaled)

In [ ]:
# Let us explore the coefficients for each of the independent attributes

for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

In [ ]:
intercept = regression_model.intercept_[0]

print("The intercept for our model is {}".format(intercept))

In [ ]:
# Model score - R2 or coeff of determinant
# R^2=1–RSS / TSS

regression_model.score(X_test_scaled, y_test_scaled)

### Observation-5:
Now we can observe by applying z score the intercept became -5.87961525130473e-16. Earlier it was -3171.950447307667.
the co-efficient has changed, the bias became nearly zero but the overall accuracy still same.

### Check Multi-collinearity using VIF

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vif = [variance_inflation_factor(X.values, ix) for ix in range(X.shape[1])] 

In [ ]:
i=0
for column in X.columns:
    if i < 11:
        print (column ,"--->",  vif[i])
        i = i+1



We can observe there are very strong multi collinearity present in the data set. Ideally it should be within 1 to 5.

#### We are exploring the Linear Regression using statsmodels as we are interested in some more statistical metrics of the model.

### Linear Regression using statsmodels.

### concatenate X and y into a single dataframe


In [ ]:
data_train = pd.concat([X_train, y_train], axis=1)
data_test=pd.concat([X_test,y_test],axis=1)
data_train.head()

In [ ]:
import statsmodels.formula.api as smf
lm1 = smf.ols(formula= 'price ~ carat+cut+color+clarity+depth+table+x+y+z', data = data_train).fit()
lm1.params

In [ ]:
print(lm1.summary())  #Inferential statistics

### Observation-6:
Assuming null  hypothesis is true, i.e there is no relationship between this variable with price. from that universe we have drawn the sample and on this sample we have found this co-efficient for the variable shown above.

Now we can ask what is the probability of finding this co-efficient in this drawn sample if in the real world the co-efficient is zero. 
As we see here the overall P value is less than alpha, so rejecting H0 and accepting Ha that atleast 1 regression co-efficient is not '0'. Here all regression co-efficients are not '0'.

For an example:
we can see the p value is showing 0.449 for 'depth' variable, which is much higher than 0.05. That means this dimension is useless.
So we can say that the attribute which are having p value greater than 0.05 are poor predictor for price.

In [ ]:
# Calculate MSE
mse = np.mean((lm1.predict(data_train.drop('price',axis=1))-data_train['price'])**2)

In [ ]:
# Calculate MSE
mse1 = np.mean((lm1.predict(data_test.drop('price',axis=1))-data_test['price'])**2)

In [ ]:
#Root Mean Squared Error - RMSE
np.sqrt(mse)

In [ ]:
#Root Mean Squared Error - RMSE
np.sqrt(mse1)

In [ ]:
# Prediction on Test data
y_pred = lm1.predict(data_test)

In [ ]:
plt.scatter(y_test['price'], y_pred)
plt.show()

In [ ]:
for i,j in np.array(lm1.params.reset_index()):
    print('({}) * {} +'.format(round(j,2),i),end=' ')

## The final Linear Regression equation is  
  
<b> price = b0 + b1 *carat[T.True] + b2 * cut + b3 * color + b4 * clarity+ b5 * depth + b6 *  table + b7 * x + b8 * y + b9 *z True </b>
  
<b> price = (-3171.95) * Intercept + (8901.94) * carat + (109.19) * cut + (272.92) * color + (436.44) * clarity + (8.24) * depth + (-17.35) * table + (-1417.91)) * x + (1464.83) * y + (-711.23) * z _True</b>  
  
1. <b>When carat increases by 1 unit, diamond price increases by 8901.94 units, keeping all other predictors constant.
2. <b>When cut increases by 1 unit, diamond price increases by 109.19 units, keeping all other predictors constant.
3. <b>When color increases by 1 unit, diamond price increases by 272.92 units, keeping all other predictors constant.
4. <b>When clarity increases by 1 unit, diamond price increases by 436.44 units, keeping all other predictors constant.
5. <b>When y increases by 1 unit, diamond price increases by 1464.83 units, keeping all other predictors constant.

<b> As per model these five attributes that are most important attributes 'Carat', 'Cut', 'color','clarity' and width i.e  <b>'y' for predicting the price.

  
  
There are also some negative co-efficient values, for instance, corresponding co-efficient (-1417.91) for 'x',(-711.23) for z and (-17.35) for table This implies, these are inversely proportional with diamond price.

### Observation-7:

* On the given data set we can see the 'X' i.e Length of the cubic zirconia in mm. having negative co-efficient. And the p value is less than 0.05, so can conclude that as higher the length of the stone is a lower profitable stones.

* Similarly for the 'z' variable
  having negative co-efficient i.e -711.23. And the p value is less than 0.05, so we can conclude that  as higher the 'z' of the stone is a lower profitable stones.
 
* Also we can see the 'y' width in mm having positive co-efficient. And the p value is less than 0.05, so we can conclude that  higher the width of the stone is a higher profitable stones.
 
* Finally we can conclude that best 5 attributes that are most important are 'Carat', 'Cut', 'color','clarity' and width i.e 'y' for predicting the price.

### 1.4 Inference: Basis on these predictions, what are the business insights and recommendations.

### Inference: 
we can see that the from the linear plot, very strong corelation between the predicted y and actual y.
But there are lots of spread. That indicates some kind noise present on the data set i.e Unexplained variances on the output.

**Linear regression Performance Metrics:**

*  intercept for the model:     -3171.950447307667
*  R square on training data: 0.9311935886926559
*  R square on testing data:   0.931543712584074
* 
*  RMSE on Training data:      907.1312415459143
*  RMSE on Testing data:        911.8447345328436


**As the training data & testing data score are almost inline,  we can conclude this model is a Right-Fit Model.**

### Impact of scaling: 
Now we can observe by applying z score the intercept became -5.87961525130473e-16. Earlier it was -3171.950447307667.
the co-efficient has changed, the bias became nearly zero but the overall accuracy still same.

### Multi collinearity: 
We can observe there are very strong multi collinearity present in the data set.

### From statsmodels:
we can see <b>R-squared:0.931 and Adj. R-squared: 0.931 are same.
The overall P value is less than alpha.

Finally we can conclude that <b>Best 5 attributes that are most important are 'Carat', 'Cut', 'color','clarity' and width i.e 'y' for predicting the price.

1. <b>When 'carat' increases by 1 unit, diamond price increases by 8901.94 units, keeping all other predictors constant.
2. <b>When 'cut' increases by 1 unit, diamond price increases by 109.19 units, keeping all other predictors constant.
3. <b>When 'color' increases by 1 unit, diamond price increases by 272.92 units, keeping all other predictors constant.
4. <b>When 'clarity' increases by 1 unit, diamond price increases by 436.44 units, keeping all other predictors constant.
5. <b>When 'y' increases by 1 unit, diamond price increases by 1464.83 units, keeping all other predictors constant.
    
1. <b>we can see the p value is showing 0.449  for depth variable, which is much greater than 0.05. 
   <b>That means this attribute is useless.



1. <b>There are also some negative co-efficient values, we can see the 'X' i.e Length of the cubic zirconia in mm. having              negative co-efficient -1417.9089.
   <b> And the p value is less than 0.05, so can conclude that as higher the length of the stone is a lower profitable stones.
    
1. <b>Similarly for the 'z' variable
   <b>having negative co-efficient i.e -711.23. And the p value is less than 0.05, so we can conclude that as higher the 'z'      <b>of the stone is a lower profitable stones.
       

### Recommendations: 

#### The Gem Stones company should consider the features'Carat', 'Cut', 'color','clarity' and width i.e 'y' as  most important for predicting the price.

To distinguish between higher profitable stones and lower profitable stones so as to have better profit share.

As we can see from the model Higher the width('y') of the stone is higher the price. 
#### So the stones having higher width('y') should consider in higher profitable stones.

The 'Premium Cut' on Diamonds are the most Expensive, followed by 'Very Good' Cut, these should consider in higher profitable stones.

The Diamonds clarity with 'VS1' &'VS2' are the most Expensive.So these two category also consider in higher profitable stones.

As we see for 'X' i.e Length of the stone, higher the length of the stone is lower the  price. 
#### So higher  the Length('x') of the stone are lower is the profitability.

higher the 'z' i.e Height of the stone is, lower the price.This is because if a Diamond's Height  is too large  Diamond will become 'Dark' in appearance because it will no longer return an Attractive amount of light. That is why
#### Stones with higher  'z'  is also are lower  in profitability.